In [4]:
import numpy as np
import argparse
import time
import cv2

import torch
from torch.autograd import Variable
import torchvision

import os
# from pytorch2caffe import pytorch2caffe, plot_graph
import numpy as np

import IPython.display as ipd

from data_utils import MudNoise
from model_stft import Mud, Mudv3, Mudv4, Mudv4noFFT, Mudv5noFFT, Mudv5LSTMnoFFT, Mudv5FFnoFFT
import json
import os
import torch
import time
from torch import nn
from torch.autograd import Variable
import csv
from ConvertModel import ConvertModel_caffe
import onnx
import caffe2.python.onnx.backend

In [2]:
# LSTM
pytorch_net = Mudv5LSTMnoFFT(n_fft=512)
dummy_input = torch.randn(1, 257, 61, 2)
torch.onnx.export(pytorch_net, dummy_input, "mudv5_lstm.onnx", verbose=False)

img = np.random.randn(1, 257, 61, 2).astype(np.float32)
# Load the ONNX model
model = onnx.load('mudv5_lstm.onnx')
outputs = caffe2.python.onnx.backend.run_model(model, [img])
print(outputs[0].shape)

NFFT IS: 512


In [16]:
# CNN
pytorch_net = Mudv5noFFT(n_fft=512, bn_ch=16, layers=6)
dummy_input = torch.randn(1, 2, 257, 61)
torch.onnx.export(pytorch_net, dummy_input, "mudv5_cnn_6_c_16.onnx", verbose=False)

img = np.random.randn(1, 2, 257, 61).astype(np.float32)
model = onnx.load('mudv5_cnn_6_c_8.onnx')
outputs = caffe2.python.onnx.backend.run_model(model, [img])
print(outputs[0].shape)

NFFT IS: 512
Receptive field TIME: 253 samples.
Receptive field FREQ: 253 samples.
(1, 257, 1, 61)


In [9]:
# FF
pytorch_net = Mudv5FFnoFFT(n_fft=512)
dummy_input = torch.randn(1, 61, 257)
torch.onnx.export(pytorch_net, dummy_input, "mudv5_ff.onnx", verbose=False)

img = np.random.randn(1, 61, 257).astype(np.float32)
model = onnx.load('mudv5_ff.onnx')
outputs = caffe2.python.onnx.backend.run_model(model, [img])
print(outputs[0].shape)

(1, 257, 1, 1, 61)


In [ ]:
# def load_mask_model(load, base_dir='.'):
#     json_dir = base_dir + '/exp/' + load
#     with open(json_dir + '/architecture.json', 'r') as fff:
#         p = json.load(fff)
#         load_path = json_dir + '/net/' + 'cv/'

#         model = Mudv5noFFT(n_fft=p['nfft'], kernel=(p['kernel1'], p['kernel2']), causal=p['causal'],
#                                 layers=p['layers'], stacks=p['stacks'], verbose=False)

#         mdl_idx = sorted([int(l.split('_')[-1].split('.')[0]) for l in os.listdir(load_path)])[-1]

#         model.load_state_dict(torch.load(load_path + 'model_weight_{}.pt'.format(mdl_idx)))
#         _ = model.eval()
#         return model, p

# pytorch_net, _ = load_mask_model('201903154220_Mudv5_512_4_NC')
pytorch_net = Mudv5noFFT(layers=6)
dummy_input = torch.randn(1, 2, 257, 61)

torch.onnx.export(pytorch_net, dummy_input, "mudv5_6l.onnx", verbose=True)

In [ ]:
print(cv2.__version__)

In [ ]:
net = cv2.dnn.readNetFromONNX('./mudv5_lstm.onnx')

In [ ]:
net.setInput(np.random.rand(1, 2, 257, 61))
st = time.time()
mask = net.forward()
print(time.time() - st)
mask.shape

st = time.time()
mask = pytorch_net(dummy_input)
print(time.time() - st)

In [ ]:
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline

causal = True
size = 3
dilation = 2 ** 3
c = nn.Conv2d(10, 10, (size, size), stride=1, padding=((dilation*(size-1)//2), (dilation*(size-1)) if causal else (dilation*(size-1)//2)),
                                dilation=dilation)

a = torch.zeros(1, 10, 257, 128)
a[:, :, 100:200, 100] = 1.

d = c(a)
d = d[:,:,:, :a.shape[-1]]
print(d.shape)

fig, ax = plt.subplots(2, 1)

ax[0].imshow(a[0, 0].data.numpy(), aspect='auto')
ax[1].imshow(d[0, 0].data.numpy(), aspect='auto')

In [ ]:
class slstm(nn.Module):
    def __init__(self, n_fft=256, hop=125, n_hid=256, n_hid_ff=513, verbose=True):
        super(slstm, self).__init__()
        if verbose:
            print("NFFT IS: {}".format(n_fft))
        self.n_fft = (n_fft // 2 + 1)
        self.FFT = n_fft
        self.HOP = hop
        
        self.lstm = nn.LSTM(input_size=self.n_fft * 2, hidden_size=n_hid)
        self.ff1 = nn.Linear(n_hid, n_hid_ff)
        self.ff2 = nn.Linear(n_hid_ff, n_hid_ff)
        self.ff3 = nn.Linear(n_hid_ff, self.n_fft)

    def forward(self, x):

#         x_stft = x.permute(0, 2, 1, 3)  # (B, L, F, 2)

#         x_input_lstm = x_stft.contiguous().view(x_stft.shape[0], x_stft.shape[1], -1)  # (B, L, F*2)

        out_lstm, _ = self.lstm(x)  # (B, L, 256)
        out_lstm2 = out_lstm.contiguous().permute(1, 0, 2)

        out1 = torch.relu(self.ff1(out_lstm2))  # (B, L, 513)
        out2 = torch.relu(self.ff2(out1))  # (B, L, 513)
        out3 = torch.sigmoid(self.ff3(out2))  # (B, L, F)

        mask_speech = out3.permute(0, 2, 1).unsqueeze(2).unsqueeze(2)  # B, F, 1, 1, T

        return mask_speech

In [ ]:
pytorch_net = slstm(n_fft=512)
dummy_input = torch.randn(10, 1, 514)

torch.onnx.export(pytorch_net, dummy_input, "mudv5_lstm_test.onnx", verbose=True)

In [ ]:
net = cv2.dnn.readNetFromONNX('./mudv5_lstm_test.onnx')

# onnx from keras

In [ ]:
max_len = 10
def RNN():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=[max_len, 256]))
    model.add(Dense(256,name='FC1', use_bias=False))
    model.add(Dense(10,name='FC2', use_bias=False))
#     inputs = Input(name='inputs',shape=[max_len, 256])
# #     layer = LSTM(64, return_sequences=True)(inputs)
#     layer = Dense(256,name='FC1', use_bias=False)(inputs)
# #     layer = Activation('relu')(layer)
# #     layer = Dropout(0.5)(layer)
#     layer = Dense(1,name='out_layer', use_bias=False)(layer)
# #     layer = Activation('sigmoid')(layer)
#     model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import keras2onnx
import onnxruntime
import onnx
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, GRU
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

# image preprocessing
# img_path = 'elephant.jpg'   # make sure the image is in img_path
# img_size = 224
# # img = image.load_img(img_path, target_size=(img_size, img_size))
# x = np.random.rand(224, 224)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# # load keras model
# model = RNN()
# model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
# print(model.summary())
# # convert to onnx model
# onnx_model = keras2onnx.convert_keras(model, model.name)

# onnx.save(onnx_model, './model_from_keras.onnx')
# # runtime prediction
# # content = onnx_model.SerializeToString()
# # sess = onnxruntime.InferenceSession(content)
# x = x if isinstance(x, list) else [x]
# feed = dict([(input.name, x[n]) for n, input in enumerate(sess.get_inputs())])
# pred_onnx = sess.run(None, feed)

In [ ]:
import numpy as np
from keras import applications
from keras import backend as K
import tensorflow as tf

K.set_learning_phase(0)  ##
model = RNN()
# model = applications.densenet.DenseNet121(input_shape=(224, 224, 3), weights='imagenet', include_top=True)
sess = K.get_session()

print(model.input, model.outputs)
print(model.summary())

In [ ]:
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

MODEL_PATH = 'out'
MODEL_NAME = 'test'
input_node_name = 'lstm_2_input'
output_node_name = 'FC2_1/MatMul'


tf.train.write_graph(sess.graph_def, MODEL_PATH, f'{MODEL_NAME}_graph.pb', as_text=False)
tf.train.write_graph(sess.graph_def, MODEL_PATH, f'{MODEL_NAME}_graph.pbtxt')
tf.train.Saver().save(sess, f'{MODEL_PATH}/{MODEL_NAME}.chkp')

freeze_graph.freeze_graph(f'{MODEL_PATH}/{MODEL_NAME}_graph.pbtxt',
                          None, False,
                          f'{MODEL_PATH}/{MODEL_NAME}.chkp',
                          output_node_name,
                          "save/restore_all",
                          "save/Const:0",
                          f'{MODEL_PATH}/frozen_{MODEL_NAME}.pb',
                          True, "")

In [ ]:
net = cv2.dnn.readNetFromONNX('./model_from_keras.onnx')
net.setInput(np.random.rand(1, 10, 256).astype('float32'))
# print(net.getFLOPS(8, (1, 10, 256)))

In [ ]:
net = cv2.dnn.readNetFromTensorflow(f'{MODEL_PATH}/frozen_{MODEL_NAME}.pb')

In [ ]:
net.setInput(np.random.rand(10, 256).astype('float32'))

In [ ]:
a = net.forward()
print(a)

# with TF

In [ ]:
from tensorflow.contrib import rnn
import tensorflow as tf
# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.LSTMBlockFusedCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)

output = tf.add(logits, 0, name='logits')


# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(tf.global_variables_initializer())
 
#Create a saver object which will save all the variables
saver = tf.train.Saver()

feed_dict ={X: np.random.rand(10, timesteps, num_input), Y: np.random.rand(10, num_classes)}
#Run the operation by feeding input
print(sess.run(logits, feed_dict))
#Prints 24 which is sum of (w1+w2)*b1 
 
#Now, save the graph
saver.save(sess, 'my_lstm_model', global_step=42)

print([n.name for n in tf.get_default_graph().as_graph_def().node])

In [ ]:
import tensorflow as tf
 
#Prepare to feed input, i.e. feed_dict and placeholders
w1 = tf.placeholder("float", name="w1")
w2 = tf.placeholder("float", name="w2")
b1= tf.Variable(2.0,name="bias")
feed_dict ={w1:4,w2:8}
 
#Define a test operation that we will restore
w3 = tf.add(w1,w2)
w4 = tf.multiply(w3,b1,name="op_to_restore")
sess = tf.Session()
sess.run(tf.global_variables_initializer())
 
#Create a saver object which will save all the variables
saver = tf.train.Saver()
 
#Run the operation by feeding input
print(sess.run(w4,feed_dict))
#Prints 24 which is sum of (w1+w2)*b1 
 
#Now, save the graph
saver.save(sess, 'my_test_model', global_step=1000)

In [ ]:
model = 'frozen_inference_graph_dnn.pb'
# net = cv2.dnn.readNet(model)
net = cv2.dnn.readNetFromTensorflow(model)

In [ ]:
import onnx
import caffe2.python.onnx.backend

# Prepare the inputs, here we use numpy to generate some random inputs for demo purpose
import numpy as np
img = np.random.randn(10, 1, 514).astype(np.float32)

# Load the ONNX model
model = onnx.load('mudv5_lstm_test.onnx')

# Run the ONNX model with Caffe2
# outputs = caffe2.python.onnx.backend.run_model(model, [img])

In [ ]:
outputs = caffe2.python.onnx.backend.run_model(model, [img])

print(outputs[0].shape)